# IMPRORT

In [1]:
from typing import Any
from py3.k_means import PixelValueGenerator, CustomKMeans
import numpy as np
from py3.CNN_model import Cplx_CustomCNN_1D
from py3.Attention_Layer import CustomAttentionLayer
from py3.classification import CustomClassifierModel
import tensorflow as tf

import re
import numpy as np
import pandas as pd

In [2]:
class Color:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    END = '\033[0m'

In [3]:
df = pd.read_csv('post_processed_data.csv')
print(df.head())


def extract_multiple_arrays(string_repr):
    # Remove letters, parentheses, spaces, and unwanted sequences
    string_repr = re.sub('[a-df-zA-DF-Z\(\)\s]', '', string_repr)
    string_repr = re.sub(r',=32', '', string_repr)

    # Find all matches of arrays within the string
    arrays = re.findall(r'\[.*?\]', string_repr)

    # Convert each found array string into a NumPy array
    #np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?', array), dtype=float) for array in arrays]
    np_arrays = [np.array(re.findall(r'[+-]?\d+(?:\.\d+)?(?:e[+-]?\d+)?', array), dtype=float) for array in arrays]
    return np_arrays


   Unnamed: 0  segment_id  polygon_id  class_id  \
0           0      367854         0.0       3.0   
1           1      367855         0.0       3.0   
2           2      367856         0.0       3.0   
3           3      369130         0.0       3.0   
4           4      363215         1.0       1.0   

                                              pixels  
0  [array([0.6406995 , 0.59669423, 0.63548833, 0....  
1  [array([4.70588237e-01, 4.76033062e-01, 4.2916...  
2  [array([0.43561208, 0.36859503, 0.3645117 , 0....  
3  [array([0.7837838 , 0.75206614, 0.5983494 , 0....  
4  [array([0.5643879 , 0.57520664, 0.4621733 , 0....  


In [ ]:
# Distribution of class_id
df['class_id'] = df['class_id'].astype('int8')
print(df['class_id'].value_counts())
 
# Plot the distribution of class_id
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.hist(df['class_id'], bins=8)
plt.title('Distribution of class_id')
plt.xlabel('class_id')
plt.ylabel('count')
plt.show()

In [4]:
# Apply the function to the 'pixels' column
df['pixels'] = df['pixels'].apply(extract_multiple_arrays)

In [5]:
# Display the resulting DataFrame
pd.set_option('display.max_colwidth', None)
print(df['pixels'].head(3))
print(len(df['pixels'].head(3)[1][5]))
# print shape
print(df['pixels'].shape)

0                                                                                                                                                                                                                                                                                                                                                                                                                              [[0.6406995, 0.59669423, 0.63548833, 0.63202727, 0.65957445, 0.49824563, 0.54585636, 0.511041, 0.52150536, 0.4963181, 0.64, 0.5751724, 0.48269582, 0.46713287, 0.4301802, 0.3372093, 0.2776747, 0.4253876, 0.5020492, 0.62374246, 0.64394546, 0.65371025, 0.6244813, 0.52614015, 0.4515464, 0.3760603, 0.34828246, 0.25601375, 0.31711555, 0.33090246, 0.34794775, 0.32139304, 0.27167115, 0.26536313, 0.23888889, 0.23178017, 0.21032746, 0.3249694, 0.29416057, 0.20978503, 0.14712153, 0.13824192, 0.1426392, 0.14430015, 0.14174253, 0.1417004, 0.15678777, 0.15600243, 0.1569966, 0.21550497, 0.2770

In [6]:
for arr_list in df['pixels']:
    for arr in arr_list:
        print(arr.shape)

(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)

In [11]:
arr_list = df['pixels'][0]
for arr in arr_list:
    print(arr.shape)
concatenated_array = np.concatenate(arr_list)
print(concatenated_array.shape)

(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(730,)
(11680,)


In [17]:
print(len(df['pixels'][0]))
arr_list = df['pixels'][0][0]
print(arr_list.shape)

stacked_array = np.vstack(df['pixels'])
print(stacked_array.shape)


16
(730,)
(49190, 730)


In [28]:
stacked_arrays = []

# Stack the arrays for each cell
for cell_pixels in df['pixels']:
    stacked_array = np.vstack(cell_pixels)
    stacked_arrays.append(stacked_array)

print(len(stacked_arrays[5][1]))
print(len(stacked_arrays))

730
3039


In [29]:
n_clusters=2
custom_kmeans = CustomKMeans(n_clusters=n_clusters)

clustered_data = []

for i in range (len(stacked_arrays)):
    custom_kmeans.fit(stacked_arrays[i])
    clustered_data.append(custom_kmeans.get_cluster_centers())
    
clusters = np.array(clustered_data)
print("clusters shape:", clusters.shape)

/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

clusters shape: (3039, 2, 73, 10)


/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/thibaultgillard/miniconda3/envs/tensorflow/lib/python3.10/si

In [30]:
print("clusters shape:", clusters.shape)

clusters shape: (3039, 2, 73, 10)
